In [8]:
from os import chdir, mkdir
from os.path import join
#the following are only available in the current development branch of IPython
from IPython.display import FileLinks, FileLink

%pylab inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Set some Pandas options
pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 25)
#populating the interactive namespace for numpy and matplotlib - not needed now...

Populating the interactive namespace from numpy and matplotlib


In [143]:
project_name = "Ashley Cohen FGLtrap"
mapping_file = "./metadata.tsv"
classifier = "./classifier_silva_132_99_16S_V4.V5_515F_926R.qza"
#to avoid changing mapping file / classifier throughout the code, just refer to it by typing $mapping_file. Use the mapping file where all columns but depth are categorical.


#NOTE: Amplicon size is ~375 bp. Reads are 2 x 250 bp.
#NOTE: the specific adapters Sabeena used exactly match Nextera Transposase Adapters The transposase adapters are used for Nextera tagmentation. Read 1 5′ TCGTCGGCAGCGTCAGATGTGTATAAGAGACAG Read 2 5′ GTCTCGTGGGCTCGGAGATGTGTATAAGAGACAG

move to file directory, use bash to unzip all fastq.gz files in all subdirectories. Use -k with gunzip to keep zipped files

In [3]:
chdir("FASTQ_Generation_2020-12-19_22_20_26Z-355373022")
!find . -type f -name '*.gz' -exec gunzip -k "{}" \;

Method 1 of doing read counts: can use one command to perform action across all subdirectories, but have to generte separate files for file name and corresponding reads this way:

get textlist of all filenames in all subdirectories

In [4]:
!find . -type f -name '*.fastq' -exec basename  {} \; >../readfnames.txt

get corresponding text list of all read counts in all subdirectories, knowing in this format 4 lines=1 read

In [5]:
!find . -type f -name '*.fastq' -exec awk '{count++} END {print count/4}'  {} \; >../readcounts.txt

Even though sequencing service performed adapter trimming, will still check for adapter contamination. <i>Can be present at 3' end</i>

Adapters as follows- unsure which is appropriate for our samples:

<b>Sequences for Nextera, Illumina Prep, and Illumina PCR Kits</b>

<u>The following sequence is used for Read 1 and Read 2 adapter trimming</u>
CTGTCTCTTATACACATCT

<u>Illumina DNA PCR-Free Prep, Tagmentation Adapter Trimming</u>
The following sequence includes two adapter sequences joined by a plus sign. When performing adapter trimming, the software independently assesses each adapter for trimming.
CTGTCTCTTATACACATCT+ATGTGTATAAGAGACA

<u>Nextera Mate Pair Adapter Trimming</u>
The following sequence includes two adapter sequences joined by a plus sign. When performing adapter trimming, the software independently assesses each adapter for trimming.
CTGTCTCTTATACACATCT+AGATGTGTATAAGAGACAG

<u>Nextera Transposase Adapters</u>
The transposase adapters are used for Nextera tagmentation. 
Read 1
5′ TCGTCGGCAGCGTCAGATGTGTATAAGAGACAG
Read 2
5′ GTCTCGTGGGCTCGGAGATGTGTATAAGAGACAG

Method 1 approach across all subdirectories:

Use grep to search for lines in each file that begin with adapter sequence since DNA reads 5' --> 3'

In [6]:
!find . -type f -name '*.fastq' -exec grep -c -e '^CTGTCTCTTATACACATCT' {} \; >../adaptcontam.txt

move copies of all individual zipped and unzipped files into respective folders

In [4]:
chdir("..")
!mkdir raw_data
!mkdir raw_zipped_data
chdir("FASTQ_Generation_2020-12-19_22_20_26Z-355373022")
!find . -mindepth 2 -type f -name '*.gz' -print -exec mv {} ../raw_zipped_data \;
!find . -mindepth 2 -type f -print -exec mv {} ../raw_data \;

mkdir: raw_data: File exists
mkdir: raw_zipped_data: File exists
./GL1-17_L001-ds.fd95d0c6e5474c869368265511c325f4/GL1-17_S7_L001_R1_001.fastq.gz
./GL1-17_L001-ds.fd95d0c6e5474c869368265511c325f4/GL1-17_S7_L001_R2_001.fastq.gz
./GEN-DONOR_L001-ds.f26c0c9e3ba74595b3004c5829eae22a/GEN-DONOR_S14_L001_R2_001.fastq.gz
./GEN-DONOR_L001-ds.f26c0c9e3ba74595b3004c5829eae22a/GEN-DONOR_S14_L001_R1_001.fastq.gz
./GL1-37_L001-ds.c07b5ea5980d4713ad6f3080f14fd948/GL1-37_S1_L001_R1_001.fastq.gz
./GL1-37_L001-ds.c07b5ea5980d4713ad6f3080f14fd948/GL1-37_S1_L001_R2_001.fastq.gz
./GL3-shallow_L001-ds.4c63cd4732874456aea31dc17fde84a4/GL3-shallow_S15_L001_R1_001.fastq.gz
./GL3-shallow_L001-ds.4c63cd4732874456aea31dc17fde84a4/GL3-shallow_S15_L001_R2_001.fastq.gz
./GL4-25_L001-ds.bfffd0dce6524291bbac7218724af67e/GL4-25_S3_L001_R2_001.fastq.gz
./GL4-25_L001-ds.bfffd0dce6524291bbac7218724af67e/GL4-25_S3_L001_R1_001.fastq.gz
./GL4-40_L001-ds.d286c42813a547a78cd2ccfe93f3f502/GL4-40_S5_L001_R1_001.fastq.gz
./GL4-40

create the directory for fast qc quality control step... Calculating the number/length of reads before quality control. 

In [7]:
!pwd

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Operation not permitted
pwd: error retrieving current directory: getcwd: cannot access parent directories: Operation not permitted
pwd: error retrieving current directory: getcwd: cannot access parent directories: Operation not permitted


go through each raw data fastq file and enumerate total low quality read flags (%) per file. Then do read counts per file. Note that the single carrot overwrites file and double carrot appends to file

In [5]:
chdir("../raw_data")
!printf 'File Name\tQuality Flag\n' > ../basequalities.txt
!for fn in *.fastq; do printf $fn; grep -o '%' $fn | wc -l;  done >> ../basequalities.txt

FileNotFoundError: [Errno 2] No such file or directory: '../raw_data'

Method 2 way of enumerating reads and looking for adapter contamination- doing once all files dumped in one directory allows file nams and reads to simultaneously be retrieved.

In [18]:
!printf 'File Name\tRead Count\n' > ../batchreadcounts.txt
!for fn in *.fastq; do printf $fn; printf '\t'; cat $fn | awk '{count++} END {print count/4}'; done >> ../batchreadcounts.txt
!printf 'File Name\tAdapter Contamination Count\n' > ../batchadaptcontam.txt
!for fn in *.fastq; do printf $fn; printf '\t'; cat $fn | grep -c -e '^CTGTCTCTTATACACATCT'; done >> ../batchadaptcontam.txt

calculate the average read length per file using awk- remember UNIX is base 1 (not base 0). Example: if NR is 1, 1-2 is -1 and -1 mod 4 is -1, which is false, and line is skipped. if NR is 2, 2-2 is 0, and 0 mod 4 is zero which is true. Using mod 4 will start at second line (sequence), then go every 4 lines.

Also find the minimum read length in each file. To do so, set variable min to the max. read length (250) and replace if length of read is less than previous min. 

In [20]:
!printf 'File Name\tMinimum Read Length\tAverage Sequence Length\n' > ../averagereadlen.txt
!for fn in *.fastq; do printf $fn; printf "\t"; awk 'BEGIN {min = 250} !((NR-2) % 4) {sum+=length($0)} !((NR-2) %4) {if (length($0) < min) min=length($0)} END {printf min; printf "\t"; printf sum/(NR/4); printf "\n"}' $fn; done >> ../averagereadlen.txt

run fastQC

In [21]:
chdir("..")
!mkdir fastqc_out
!fastqc -t 4 raw_data/* -o fastqc_outraw

Started analysis of GEN-DONOR_S14_L001_R1_001.fastq
Approx 5% complete for GEN-DONOR_S14_L001_R1_001.fastq
Approx 10% complete for GEN-DONOR_S14_L001_R1_001.fastq
Started analysis of GEN-DONOR_S14_L001_R2_001.fastq
Approx 15% complete for GEN-DONOR_S14_L001_R1_001.fastq
Approx 5% complete for GEN-DONOR_S14_L001_R2_001.fastq
Approx 20% complete for GEN-DONOR_S14_L001_R1_001.fastq
Approx 10% complete for GEN-DONOR_S14_L001_R2_001.fastq
Approx 25% complete for GEN-DONOR_S14_L001_R1_001.fastq
Approx 15% complete for GEN-DONOR_S14_L001_R2_001.fastq
Started analysis of GL1-17_S7_L001_R1_001.fastq
Approx 30% complete for GEN-DONOR_S14_L001_R1_001.fastq
Approx 5% complete for GL1-17_S7_L001_R1_001.fastq
Approx 20% complete for GEN-DONOR_S14_L001_R2_001.fastq
Approx 10% complete for GL1-17_S7_L001_R1_001.fastq
Approx 25% complete for GEN-DONOR_S14_L001_R2_001.fastq
Approx 15% complete for GL1-17_S7_L001_R1_001.fastq
Approx 35% complete for GEN-DONOR_S14_L001_R1_001.fastq
Approx 40% complete for

Approx 55% complete for GL2-17_S11_L001_R2_001.fastq
Approx 95% complete for GL2-17_S11_L001_R1_001.fastq
Approx 60% complete for GL2-17_S11_L001_R2_001.fastq
Analysis complete for GL2-17_S11_L001_R1_001.fastq
Approx 65% complete for GL2-17_S11_L001_R2_001.fastq
Approx 70% complete for GL2-17_S11_L001_R2_001.fastq
Approx 75% complete for GL2-17_S11_L001_R2_001.fastq
Approx 80% complete for GL2-17_S11_L001_R2_001.fastq
Approx 85% complete for GL2-17_S11_L001_R2_001.fastq
Approx 90% complete for GL2-17_S11_L001_R2_001.fastq
Approx 95% complete for GL2-17_S11_L001_R2_001.fastq
Analysis complete for GL2-17_S11_L001_R2_001.fastq
Started analysis of GL2-23_S6_L001_R1_001.fastq
Approx 5% complete for GL2-23_S6_L001_R1_001.fastq
Approx 10% complete for GL2-23_S6_L001_R1_001.fastq
Approx 15% complete for GL2-23_S6_L001_R1_001.fastq
Approx 20% complete for GL2-23_S6_L001_R1_001.fastq
Approx 25% complete for GL2-23_S6_L001_R1_001.fastq
Approx 30% complete for GL2-23_S6_L001_R1_001.fastq
Started a

Approx 35% complete for GL3-mid_S13_L001_R2_001.fastq
Approx 85% complete for GL3-mid_S13_L001_R1_001.fastq
Approx 95% complete for GL3-deep_S2_L001_R2_001.fastq
Approx 40% complete for GL3-mid_S13_L001_R2_001.fastq
Approx 90% complete for GL3-mid_S13_L001_R1_001.fastq
Analysis complete for GL3-deep_S2_L001_R2_001.fastq
Approx 45% complete for GL3-mid_S13_L001_R2_001.fastq
Approx 95% complete for GL3-mid_S13_L001_R1_001.fastq
Analysis complete for GL3-mid_S13_L001_R1_001.fastq
Approx 50% complete for GL3-mid_S13_L001_R2_001.fastq
Approx 55% complete for GL3-mid_S13_L001_R2_001.fastq
Approx 60% complete for GL3-mid_S13_L001_R2_001.fastq
Approx 65% complete for GL3-mid_S13_L001_R2_001.fastq
Approx 70% complete for GL3-mid_S13_L001_R2_001.fastq
Approx 75% complete for GL3-mid_S13_L001_R2_001.fastq
Approx 80% complete for GL3-mid_S13_L001_R2_001.fastq
Approx 85% complete for GL3-mid_S13_L001_R2_001.fastq
Approx 90% complete for GL3-mid_S13_L001_R2_001.fastq
Approx 95% complete for GL3-mid_

Approx 50% complete for GL4-40_S5_L001_R1_001.fastq
Approx 95% complete for GL4-25_S3_L001_R2_001.fastq
Approx 55% complete for GL4-40_S5_L001_R1_001.fastq
Analysis complete for GL4-25_S3_L001_R2_001.fastq
Approx 60% complete for GL4-40_S5_L001_R1_001.fastq
Approx 65% complete for GL4-40_S5_L001_R1_001.fastq
Approx 70% complete for GL4-40_S5_L001_R1_001.fastq
Approx 75% complete for GL4-40_S5_L001_R1_001.fastq
Started analysis of GL4-40_S5_L001_R2_001.fastq
Approx 80% complete for GL4-40_S5_L001_R1_001.fastq
Approx 5% complete for GL4-40_S5_L001_R2_001.fastq
Approx 10% complete for GL4-40_S5_L001_R2_001.fastq
Approx 85% complete for GL4-40_S5_L001_R1_001.fastq
Approx 15% complete for GL4-40_S5_L001_R2_001.fastq
Approx 90% complete for GL4-40_S5_L001_R1_001.fastq
Approx 20% complete for GL4-40_S5_L001_R2_001.fastq
Approx 95% complete for GL4-40_S5_L001_R1_001.fastq
Approx 25% complete for GL4-40_S5_L001_R2_001.fastq
Analysis complete for GL4-40_S5_L001_R1_001.fastq
Approx 30% complete f

Approx 70% complete for tr-blk_S9_L001_R2_001.fastq
Approx 75% complete for tr-blk_S9_L001_R2_001.fastq
Approx 80% complete for tr-blk_S9_L001_R2_001.fastq
Approx 85% complete for tr-blk_S9_L001_R2_001.fastq
Approx 90% complete for tr-blk_S9_L001_R2_001.fastq
Approx 95% complete for tr-blk_S9_L001_R2_001.fastq
Analysis complete for tr-blk_S9_L001_R2_001.fastq


create directory for reads, set sample data type, which can be, for FASTQ (i.e., sequence data with quality information):

-FASTQ data with the EMP Protocol format

-Multiplexed FASTQ data with barcodes in sequences

-FASTQ data in the Casava 1.8 demultiplexed format

-Any other FASTQ

...for more information see https://docs.qiime2.org/2020.11/tutorials/importing/#sequence-data-with-sequence-quality-information-i-e-fastq

In [14]:
!pwd

/Users/ashley/Documents/Research/Gordon'sLab/FGL/amplicon/Qiime2_analyzed_FGL_traps


In [15]:
!mkdir reads_qza

!qiime tools import --type SampleData[PairedEndSequencesWithQuality] \
                   --input-path raw_zipped_data \
                   --output-path reads_qza/reads.qza \
                   --input-format CasavaOneEightSingleLanePerSampleDirFmt

Imported raw_zipped_data as CasavaOneEightSingleLanePerSampleDirFmt to reads_qza/reads.qza


Barcode extraction does not need to be done for these samples

Trim primers with cutadapt - remove all primers and take out all sequences that don't begin with priemr sequence 16S V4-V5 region bacteria + archaea F = GTGYCAGCMGCCGCGGTAA R = GGACTACNVGGGTWTCTAAT ...ask Jacob for primer names and lit references. The ones i am used to are 515F - GTGYCAGCMGCCGCGGTAA 926R - CCGYCAATTYMTTTRAGTTT 

In [16]:
!qiime cutadapt trim-paired --i-demultiplexed-sequences reads_qza/reads.qza \
                           --p-cores 4 \
                           --p-front-f ^GTGYCAGCMGCCGCGGTAA \
                           --p-front-r ^CCGYCAATTYMTTTRAGTTT \
                           --o-trimmed-sequences reads_qza/reads_trimmed.qza

Saved SampleData[PairedEndSequencesWithQuality] to: reads_qza/reads_trimmed.qza


according to fastqc, cut-offs for best quality data should be 185 and 160, but Jacob said: f and r reads won't overlap so have to compromise w 230 and 210, and needed to set ee-f and ee-r to 3 and 4, respectively.

In [100]:
!qiime dada2 denoise-paired --i-demultiplexed-seqs reads_qza/reads_trimmed.qza \
                           --p-trunc-len-f 205 \
                           --p-trunc-len-r 185 \
                           --p-max-ee-f 3 \
                           --p-max-ee-r 3 \
                           --p-n-threads 4 \
                           --output-dir dada2_output

Saved FeatureTable[Frequency] to: dada2_output/table.qza
Saved FeatureData[Sequence] to: dada2_output/representative_sequences.qza
Saved SampleData[DADA2Stats] to: dada2_output/denoising_stats.qza


input the feature/frequency table of trimmed/denoise reads from the original pipeline output, then filter feature/frequency table by minimum frequency and number of occurrences- save as "with chloromito" so have those sequences after filtration later.

In [101]:
!qiime feature-table filter-features --i-table dada2_output/table.qza \
                                    --p-min-frequency 20 \
                                    --p-min-samples 1 \
                                    --o-filtered-table dada2_output/table_filt_wchloromito.qza

Saved FeatureTable[Frequency] to: dada2_output/table_filt_wchloromito.qza


input the new filtered feature/frequency table and the feature/sequence table from original pipeline output to make filtered feature/sequence table. Representative sequences comes from the original pipeline (after trimming/denoising). Save as "with chloromito" so don't overwrite later

In [102]:
!qiime feature-table filter-seqs --i-data dada2_output/representative_sequences.qza \
                                --i-table dada2_output/table_filt_wchloromito.qza \
                                --o-filtered-data dada2_output/rep_seqs_filt_wchloromito.qza

Saved FeatureData[Sequence] to: dada2_output/rep_seqs_filt_wchloromito.qza


denoising stats will be exported as .tsv, rep_seqs_filt will be exported as fasta, and table_filt (the classic OTU table) will be exported as .biom file and need to be converted to .tsv format using biom convert. I am also exporting the unfiltered rep seqs fasta to look in it for the spike. Save exported repseqs and exp filtered table as w chloromito to save if want to look for those things, or will be overwritten later.

In [103]:
!qiime tools export --input-path dada2_output/denoising_stats.qza --output-path dada2_output/exported_denoising_stats
!qiime tools export --input-path dada2_output/rep_seqs_filt_wchloromito.qza --output-path dada2_output/exported_rep_seq_filt_wchloromito
!qiime tools export --input-path dada2_output/table_filt_wchloromito.qza --output-path dada2_output/exported_table_filt_wchloromito
!qiime tools export --input-path dada2_output/representative_sequences.qza --output-path dada2_output/exported_representative_sequences

!biom convert -i dada2_output/exported_table_filt_wchloromito/feature-table.biom -o dada2_output/exported_table_filt_wchloromito/exp_table_filtwchloromito.tsv --to-tsv

Exported dada2_output/denoising_stats.qza as DADA2StatsDirFmt to directory dada2_output/exported_denoising_stats
Exported dada2_output/rep_seqs_filt_wchloromito.qza as DNASequencesDirectoryFormat to directory dada2_output/exported_rep_seq_filt_wchloromito
Exported dada2_output/table_filt_wchloromito.qza as BIOMV210DirFmt to directory dada2_output/exported_table_filt_wchloromito
Exported dada2_output/representative_sequences.qza as DNASequencesDirectoryFormat to directory dada2_output/exported_representative_sequences


 create visualizations of completed dada2 outputs

In [104]:
!mkdir reports

!qiime metadata tabulate --m-input-file dada2_output/denoising_stats.qza --o-visualization reports/stats_dada2.qzv
!qiime feature-table summarize --i-table dada2_output/table_filt_wchloromito.qza --o-visualization reports/table_filt_wchloromito.qzv
!qiime feature-table summarize --i-table dada2_output/table.qza --o-visualization reports/table.qzv
!qiime feature-table tabulate-seqs --i-data dada2_output/representative_sequences.qza --o-visualization reports/representative_sequences.qzv
!qiime feature-table tabulate-seqs --i-data dada2_output/rep_seqs_filt_wchloromito.qza --o-visualization reports/rep_seqs_filt_wchloromito.qzv

Saved Visualization to: reports/stats_dada2.qzv
Saved Visualization to: reports/table_filt_wchloromito.qzv
Saved Visualization to: reports/table.qzv
Saved Visualization to: reports/representative_sequences.qzv
Saved Visualization to: reports/rep_seqs_filt_wchloromito.qzv


In [22]:
!pwd

/Users/ashley/Documents/Research/Gordon'sLab/FGL/amplicon/Qiime2_analyzed_FGL_traps


In [105]:
!qiime feature-classifier classify-sklearn --i-reads dada2_output/rep_seqs_filt_wchloromito.qza \
                                          --i-classifier $classifier \
                                          --p-n-jobs -2 \
                                          --output-dir taxa_wchloromito  --verbose

Saved FeatureData[Taxonomy] to: taxa_wchloromito/classification.qza


In [106]:
!qiime tools export --input-path taxa_wchloromito/classification.qza --output-path taxa_wchloromito

Exported taxa_wchloromito/classification.qza as TSVTaxonomyDirectoryFormat to directory taxa_wchloromito


input the filtered feature/frequency and taxonomy classifier and exclude chloroplasts and mitochondria, then save as table_filt.

In [107]:
!qiime taxa filter-table --i-table dada2_output/table_filt_wchloromito.qza \
                        --i-taxonomy taxa_wchloromito/classification.qza \
                        --p-exclude Chloroplast,chloroplast,mitochondria,Mitochondria \
                        --o-filtered-table dada2_output/table_filt.qza

Saved FeatureTable[Frequency] to: dada2_output/table_filt.qza


Once there is a new filtered table, the representative sequences table needs to be re-filtered to reflect the new frequency/feature table. input the feature/sequence and filtered feature/frequency table to make a filtered feature/sequence table. Then reclassify.

In [108]:
!qiime feature-table filter-seqs --i-data dada2_output/representative_sequences.qza \
                                --i-table dada2_output/table_filt.qza \
                                --o-filtered-data dada2_output/rep_seqs_filt.qza

Saved FeatureData[Sequence] to: dada2_output/rep_seqs_filt.qza


In [109]:
!qiime feature-classifier classify-sklearn --i-reads dada2_output/rep_seqs_filt.qza \
                                          --i-classifier $classifier \
                                          --p-n-jobs -2 \
                                          --output-dir taxa  --verbose

Saved FeatureData[Taxonomy] to: taxa/classification.qza


Export all the chloroplast and mito. free files.

In [110]:
!qiime tools export --input-path taxa/classification.qza --output-path taxa
!qiime tools export --input-path dada2_output/rep_seqs_filt.qza --output-path dada2_output/exported_rep_seq_filt
!qiime tools export --input-path dada2_output/table_filt.qza --output-path dada2_output/exported_table_filt

!biom convert -i dada2_output/exported_table_filt/feature-table.biom -o dada2_output/exported_table_filt/table_filt.tsv --to-tsv


Exported taxa/classification.qza as TSVTaxonomyDirectoryFormat to directory taxa
Exported dada2_output/rep_seqs_filt.qza as DNASequencesDirectoryFormat to directory dada2_output/exported_rep_seq_filt
Exported dada2_output/table_filt.qza as BIOMV210DirFmt to directory dada2_output/exported_table_filt


export all the visualizations to reports

In [111]:
!qiime feature-table summarize --i-table dada2_output/table_filt.qza --o-visualization reports/table_filt.qzv
!qiime feature-table tabulate-seqs --i-data dada2_output/rep_seqs_filt.qza --o-visualization reports/rep_seqs_filt.qzv

Saved Visualization to: reports/table_filt.qzv
Saved Visualization to: reports/rep_seqs_filt.qzv


# Build quick phylogeny with FastTree
make multiple-sequence alignment using MAFFT. Export the filtered aligned masked sequences as an aligned fasta file. 

In [112]:
!mkdir tree_out

In [113]:
!qiime alignment mafft --i-sequences dada2_output/rep_seqs_filt.qza \
                      --p-n-threads 4 \
                      --o-alignment tree_out/rep_seqs_filt_aligned.qza

Saved FeatureData[AlignedSequence] to: tree_out/rep_seqs_filt_aligned.qza


In [114]:
!qiime alignment mask --i-alignment tree_out/rep_seqs_filt_aligned.qza \
  --o-masked-alignment tree_out/rep_seqs_filt_aligned_masked.qza

Saved FeatureData[AlignedSequence] to: tree_out/rep_seqs_filt_aligned_masked.qza


In [115]:
!qiime tools export --input-path tree_out/rep_seqs_filt_aligned_masked.qza --output-path tree_out
chdir("tree_out")
#!ls
!mv aligned-dna-sequences.fasta aligned_masked_rep_seqs_filt.fasta
chdir("..")

Exported tree_out/rep_seqs_filt_aligned_masked.qza as AlignedDNASequencesDirectoryFormat to directory tree_out


## Running FastTree
Finally FastTree can be run on this masked multiple-sequence alignment. Export the rooted tree as Newick string:

In [116]:
!qiime phylogeny fasttree --i-alignment tree_out/rep_seqs_filt_aligned_masked.qza \
                         --p-n-threads 4 \
                         --o-tree tree_out/rep_seqs_filt_aligned_masked_tree

Saved Phylogeny[Unrooted] to: tree_out/rep_seqs_filt_aligned_masked_tree.qza


Add root to tree...
FastTree returns an unrooted tree. One basic way to add a root to a tree is to add it add it at the midpoint of the largest tip-to-tip distance in the tree, which is done with this command:

In [117]:
!qiime phylogeny midpoint-root --i-tree tree_out/rep_seqs_filt_aligned_masked_tree.qza \
                              --o-rooted-tree tree_out/rep_seqs_filt_aligned_masked_tree_rooted.qza

Saved Phylogeny[Rooted] to: tree_out/rep_seqs_filt_aligned_masked_tree_rooted.qza


In [118]:
!qiime tools export --input-path tree_out/rep_seqs_filt_aligned_masked_tree_rooted.qza --output-path tree_out

Exported tree_out/rep_seqs_filt_aligned_masked_tree_rooted.qza as NewickDirectoryFormat to directory tree_out


Generate rarefaction curves A key quality control step is to plot rarefaction curves for all of your samples to determine if you performed sufficient sequencing. The below command will generate these plots (X is a placeholder for the maximum depth in your dataset, i.e., minimum feature frequency per sample- which you can determine by running the summarize command above).
## remember to change x -- p-max-depth

In [144]:
!qiime diversity alpha-rarefaction \
    --i-table dada2_output/table_filt.qza \
    --p-max-depth 20031 \
    --p-steps 20 \
    --i-phylogeny tree_out/rep_seqs_filt_aligned_masked_tree_rooted.qza \
    --m-metadata-file $mapping_file \
    --o-visualization rarefaction_curves.qzv

Saved Visualization to: rarefaction_curves.qzv


rarefaction OTU table simulations- produce OTU tables at varying p-max depths to see differences in community composition. ...j/k- only works in QIIME1.

"To perform bootstrap, jackknife, and rarefaction analyses, the otu table must be subsampled (rarefied). This script rarefies, or subsamples, OTU tables. This does not provide curves of diversity by number of sequences in a sample. Rather it creates a series of subsampled OTU tables by random sampling (without replacement) of the input OTU table. Samples that have fewer sequences then the requested rarefaction depth for a given output otu table are omitted from those ouput otu tables. The pseudo-random number generator used for rarefaction by subsampling is NumPy’s default - an implementation of the Mersenne twister PRNG."

In [50]:
#multiple_rarefactions.py -i dada2_output/exported_table_filt/feature-table.biom -m 4000 -x 18100 -s 100 -n 10 -o rare_4000-18100/

Error: QIIME 2 has no plugin/command named 'multiple_rarefactions'.


## Assign taxonomy
You can assign taxonomy to your ASVs using a Naive-Bayes approach implemented in the scikit learn Python library and the SILVA database. Note that we have trained classifiers for a few different amplicon regions already (which are available in the /home/shared/taxa_classifiers folder), but you will need to generate your own if your region of interest isn't there. The classifier filename below is for the V6/V8 B969F-BA1406R region. The regions that we have trained classifiers for are:

A more useful output is the interactive stacked bar-charts of the taxonomic abundances across samples, which can be output with this command:

As with all QZA files, you can export the output file to take a look at the classifications and confidence scores:

A more useful output is the interactive stacked bar-charts of the taxonomic abundances across samples, which can be output with this command:

In [145]:
!qiime taxa barplot --i-table dada2_output/table_filt.qza \
                   --i-taxonomy taxa/classification.qza \
                   --m-metadata-file $mapping_file \
                   --o-visualization taxa/taxa_barplot.qzv

Saved Visualization to: taxa/taxa_barplot.qzv


Calculating diversity metrics and generating ordination plots
Common alpha and beta-diversity metrics can be calculated with a single command in QIIME2. In addition, ordination plots (such as PCoA plots for weighted UniFrac distances) will be generated automatically as well. This command will also rarefy all samples to the sample sequencing depth before calculating these metrics (X is a placeholder for the lowest reasonable sample depth; samples with depth below this cut-off will be excluded).

In [146]:
!qiime diversity core-metrics-phylogenetic --i-table dada2_output/table_filt.qza \
                                          --i-phylogeny tree_out/rep_seqs_filt_aligned_masked_tree_rooted.qza \
                                          --p-sampling-depth 20031 \
                                          --m-metadata-file $mapping_file \
                                          --p-n-jobs 2 \
                                          --output-dir diversity2

Saved FeatureTable[Frequency] to: diversity2/rarefied_table.qza
Saved SampleData[AlphaDiversity] % Properties('phylogenetic') to: diversity2/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: diversity2/observed_otus_vector.qza
Saved SampleData[AlphaDiversity] to: diversity2/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: diversity2/evenness_vector.qza
Saved DistanceMatrix % Properties('phylogenetic') to: diversity2/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix % Properties('phylogenetic') to: diversity2/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: diversity2/jaccard_distance_matrix.qza
Saved DistanceMatrix to: diversity2/bray_curtis_distance_matrix.qza
Saved PCoAResults to: diversity2/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: diversity2/weighted_unifrac_pcoa_results.qza
Saved PCoAResults to: diversity2/jaccard_pcoa_results.qza
Saved PCoAResults to: diversity2/bray_curtis_pcoa_results.qza
Saved Visualization to: diversity2/un

Output diversity2 qzas to tsvs.

In [147]:
chdir("diversity2")

!qiime tools export --input-path bray_curtis_distance_matrix.qza --output-path temp
!mv temp/distance-matrix.tsv braydistance-matrix.tsv

!qiime tools export --input-path jaccard_distance_matrix.qza --output-path temp
!mv temp/distance-matrix.tsv jaccarddist-matrix.tsv

!qiime tools export --input-path unweighted_unifrac_distance_matrix.qza --output-path temp
!mv temp/distance-matrix.tsv unweightunifr.tsv

!qiime tools export --input-path weighted_unifrac_distance_matrix.qza --output-path temp
!mv temp/distance-matrix.tsv weightunifr-matrix.tsv

!qiime tools export --input-path evenness_vector.qza --output-path temp
!mv temp/alpha-diversity.tsv evenessalpha.tsv

!qiime tools export --input-path faith_pd_vector.qza --output-path temp
!mv temp/alpha-diversity.tsv faithpdalpha.tsv

!qiime tools export --input-path rarefied_table.qza --output-path temp #biom
!biom convert -i temp/feature-table.biom -o rarifiedtable.tsv --to-tsv

!qiime tools export --input-path observed_otus_vector.qza --output-path temp
!mv temp/alpha-diversity.tsv observedOTUs.tsv

!qiime tools export --input-path shannon_vector.qza --output-path temp
!mv temp/alpha-diversity.tsv shannonvector.tsv

!rm -r temp

chdir("..")

Exported bray_curtis_distance_matrix.qza as DistanceMatrixDirectoryFormat to directory temp
Exported jaccard_distance_matrix.qza as DistanceMatrixDirectoryFormat to directory temp
Exported unweighted_unifrac_distance_matrix.qza as DistanceMatrixDirectoryFormat to directory temp
Exported weighted_unifrac_distance_matrix.qza as DistanceMatrixDirectoryFormat to directory temp
Exported evenness_vector.qza as AlphaDiversityDirectoryFormat to directory temp
Exported faith_pd_vector.qza as AlphaDiversityDirectoryFormat to directory temp
Exported rarefied_table.qza as BIOMV210DirFmt to directory temp
Exported observed_otus_vector.qza as AlphaDiversityDirectoryFormat to directory temp
Exported shannon_vector.qza as AlphaDiversityDirectoryFormat to directory temp


Identifying differentially abundant features with ANCOM
ANCOM is one method to test for difference in the relative abundance of features between sample groupings. 
It is a compositional approach that makes no assumptions about feature distributions. 

... basically, we want to find stuff that plots near top right as being truly different across groups

However, it requires that all features have non-zero abundances so a pseudocount of 1 first needs to be added:

In [123]:
!qiime composition add-pseudocount --i-table dada2_output/table_filt.qza \
                                  --o-composition-table dada2_output/table_filt_pseudocount.qza

Saved FeatureTable[Composition] to: dada2_output/table_filt_pseudocount.qza


run ancom for month variable...

In [140]:
!qiime composition ancom --i-table dada2_output/table_filt_pseudocount.qza \
                        --m-metadata-file $mapping_file \
                        --m-metadata-column Month \
                        --output-dir ancom_DEG_output_month

Saved Visualization to: ancom_DEG_output_month/visualization.qzv


run ancom for pos variable...

In [148]:
!qiime composition ancom --i-table dada2_output/table_filt_pseudocount.qza \
                        --m-metadata-file $mapping_file \
                        --m-metadata-column pos \
                        --output-dir ancom_DEG_output_pos

Saved Visualization to: ancom_DEG_output_pos/visualization.qzv


run ancom for depth variable...

In [ ]:
!qiime composition ancom --i-table dada2_output/table_filt_pseudocount.qza \
                        --m-metadata-file $mapping_file \
                        --m-metadata-column Depth \
                        --output-dir ancom_DEG_output_Depth

In [92]:
#!qiime tools export --input-path dada2_output/table_filt.qza --output-path dada2_output_exported
#!qiime tools export --input-path dada2_output/rep_seqs_filt.qza --output-path dada2_output_exported

Exported dada2_output/table_filt.qza as BIOMV210DirFmt to directory dada2_output_exported
Exported dada2_output/rep_seqs_filt.qza as DNASequencesDirectoryFormat to directory dada2_output_exported


In [149]:
!qiime diversity alpha-group-significance \
  --i-alpha-diversity diversity2/faith_pd_vector.qza \
  --m-metadata-file $mapping_file \
  --o-visualization diversity2/faith-pd-group-significance.qzv

!qiime diversity alpha-group-significance \
  --i-alpha-diversity diversity2/evenness_vector.qza \
  --m-metadata-file $mapping_file \
  --o-visualization diversity2/evenness-group-significance.qzv

Saved Visualization to: diversity2/faith-pd-group-significance.qzv
Saved Visualization to: diversity2/evenness-group-significance.qzv


In [150]:
!qiime diversity beta-group-significance \
  --i-distance-matrix diversity2/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file $mapping_file \
  --m-metadata-column Month \
  --o-visualization diversity2/unweighted-unifrac-Month-group-significance.qzv \
  --p-pairwise

Saved Visualization to: diversity2/unweighted-unifrac-Month-group-significance.qzv


produce all of the "classic OTU tables" with frequencies and taxonomy at desired taxonomic levels by inputting the filtered feature frequency table and taxonomy classification and using taxa-collapse. Also output additional pseudocount and ancom data at finest level. **NOTE the default output level for QIIME2 is 3 (class). If you need other levels, specify.**

In [128]:
!mkdir abs_OTUtables

In [129]:
!qiime taxa collapse \
  --i-table dada2_output/table_filt.qza \
  --i-taxonomy taxa/classification.qza \
  --p-level 7 \
  --o-collapsed-table abs_OTUtables/table_filt-16_L7.qza

!qiime taxa collapse \
  --i-table dada2_output/table_filt.qza \
  --i-taxonomy taxa/classification.qza \
  --p-level 2 \
  --o-collapsed-table abs_OTUtables/table_filt-16_L2.qza

!qiime taxa collapse \
  --i-table dada2_output/table_filt.qza \
  --i-taxonomy taxa/classification.qza \
  --p-level 3 \
  --o-collapsed-table abs_OTUtables/table_filt-16_L3.qza

!qiime taxa collapse \
  --i-table dada2_output/table_filt.qza \
  --i-taxonomy taxa/classification.qza \
  --p-level 4 \
  --o-collapsed-table abs_OTUtables/table_filt-16_L4.qza

!qiime taxa collapse \
  --i-table dada2_output/table_filt.qza \
  --i-taxonomy taxa/classification.qza \
  --p-level 5 \
  --o-collapsed-table abs_OTUtables/table_filt-16_L5.qza

!qiime taxa collapse \
  --i-table dada2_output/table_filt.qza \
  --i-taxonomy taxa/classification.qza \
  --p-level 6 \
  --o-collapsed-table abs_OTUtables/table_filt-16_L6.qza

Saved FeatureTable[Frequency] to: abs_OTUtables/table_filt-16_L7.qza
Saved FeatureTable[Frequency] to: abs_OTUtables/table_filt-16_L2.qza
Saved FeatureTable[Frequency] to: abs_OTUtables/table_filt-16_L3.qza
Saved FeatureTable[Frequency] to: abs_OTUtables/table_filt-16_L4.qza
Saved FeatureTable[Frequency] to: abs_OTUtables/table_filt-16_L5.qza
Saved FeatureTable[Frequency] to: abs_OTUtables/table_filt-16_L6.qza


export the taxa-collapse qza files to temporary directories, then convert biom files to tsv.

In [130]:
!qiime tools export --input-path abs_OTUtables/table_filt-16_L7.qza --output-path roo1
!qiime tools export --input-path abs_OTUtables/table_filt-16_L2.qza --output-path roo2
!qiime tools export --input-path abs_OTUtables/table_filt-16_L3.qza --output-path roo3
!qiime tools export --input-path abs_OTUtables/table_filt-16_L4.qza --output-path roo4
!qiime tools export --input-path abs_OTUtables/table_filt-16_L5.qza --output-path roo5
!qiime tools export --input-path abs_OTUtables/table_filt-16_L6.qza --output-path roo6

Exported abs_OTUtables/table_filt-16_L7.qza as BIOMV210DirFmt to directory roo1
Exported abs_OTUtables/table_filt-16_L2.qza as BIOMV210DirFmt to directory roo2
Exported abs_OTUtables/table_filt-16_L3.qza as BIOMV210DirFmt to directory roo3
Exported abs_OTUtables/table_filt-16_L4.qza as BIOMV210DirFmt to directory roo4
Exported abs_OTUtables/table_filt-16_L5.qza as BIOMV210DirFmt to directory roo5
Exported abs_OTUtables/table_filt-16_L6.qza as BIOMV210DirFmt to directory roo6


In [131]:
!biom convert -i roo1/feature-table.biom -o abs_OTUtables/table_filt-16_L7.tsv --to-tsv
!biom convert -i roo2/feature-table.biom -o abs_OTUtables/table_filt-16_L2.tsv --to-tsv
!biom convert -i roo3/feature-table.biom -o abs_OTUtables/table_filt-16_L3.tsv --to-tsv
!biom convert -i roo4/feature-table.biom -o abs_OTUtables/table_filt-16_L4.tsv --to-tsv
!biom convert -i roo5/feature-table.biom -o abs_OTUtables/table_filt-16_L5.tsv --to-tsv
!biom convert -i roo6/feature-table.biom -o abs_OTUtables/table_filt-16_L6.tsv --to-tsv

use the taxa collapsed frequency/OTU tables to create relative abundance tables, export to biom, convert biom to tsv. Then remove the temporary directories.

In [132]:
!mkdir rel_OTUtables

In [133]:
!qiime feature-table relative-frequency \
--i-table abs_OTUtables/table_filt-16_L7.qza \
--o-relative-frequency-table rel_OTUtables/rel-phyla-table_L7.qza

!qiime feature-table relative-frequency \
--i-table abs_OTUtables/table_filt-16_L3.qza \
--o-relative-frequency-table rel_OTUtables/rel-phyla-table_L3.qza

!qiime feature-table relative-frequency \
--i-table abs_OTUtables/table_filt-16_L2.qza \
--o-relative-frequency-table rel_OTUtables/rel-phyla-table_L2.qza

!qiime feature-table relative-frequency \
--i-table abs_OTUtables/table_filt-16_L4.qza \
--o-relative-frequency-table rel_OTUtables/rel-phyla-table_L4.qza

!qiime feature-table relative-frequency \
--i-table abs_OTUtables/table_filt-16_L5.qza \
--o-relative-frequency-table rel_OTUtables/rel-phyla-table_L5.qza

!qiime feature-table relative-frequency \
--i-table abs_OTUtables/table_filt-16_L6.qza \
--o-relative-frequency-table rel_OTUtables/rel-phyla-table_L6.qza

!qiime tools export --input-path rel_OTUtables/rel-phyla-table_L7.qza --output-path roo1
!qiime tools export --input-path rel_OTUtables/rel-phyla-table_L2.qza --output-path roo2
!qiime tools export --input-path rel_OTUtables/rel-phyla-table_L3.qza --output-path roo3
!qiime tools export --input-path rel_OTUtables/rel-phyla-table_L4.qza --output-path roo4
!qiime tools export --input-path rel_OTUtables/rel-phyla-table_L5.qza --output-path roo5
!qiime tools export --input-path rel_OTUtables/rel-phyla-table_L6.qza --output-path roo6

!biom convert -i roo1/feature-table.biom -o rel_OTUtables/rel-phyla-table_L7.tsv --to-tsv
!biom convert -i roo2/feature-table.biom -o rel_OTUtables/rel-phyla-table_L2.tsv --to-tsv
!biom convert -i roo3/feature-table.biom -o rel_OTUtables/rel-phyla-table_L3.tsv --to-tsv
!biom convert -i roo4/feature-table.biom -o rel_OTUtables/rel-phyla-table_L4.tsv --to-tsv
!biom convert -i roo5/feature-table.biom -o rel_OTUtables/rel-phyla-table_L5.tsv --to-tsv
!biom convert -i roo6/feature-table.biom -o rel_OTUtables/rel-phyla-table_L6.tsv --to-tsv

Saved FeatureTable[RelativeFrequency] to: rel_OTUtables/rel-phyla-table_L7.qza
Saved FeatureTable[RelativeFrequency] to: rel_OTUtables/rel-phyla-table_L3.qza
Saved FeatureTable[RelativeFrequency] to: rel_OTUtables/rel-phyla-table_L2.qza
Saved FeatureTable[RelativeFrequency] to: rel_OTUtables/rel-phyla-table_L4.qza
Saved FeatureTable[RelativeFrequency] to: rel_OTUtables/rel-phyla-table_L5.qza
Saved FeatureTable[RelativeFrequency] to: rel_OTUtables/rel-phyla-table_L6.qza
Exported rel_OTUtables/rel-phyla-table_L7.qza as BIOMV210DirFmt to directory roo1
Exported rel_OTUtables/rel-phyla-table_L2.qza as BIOMV210DirFmt to directory roo2
Exported rel_OTUtables/rel-phyla-table_L3.qza as BIOMV210DirFmt to directory roo3
Exported rel_OTUtables/rel-phyla-table_L4.qza as BIOMV210DirFmt to directory roo4
Exported rel_OTUtables/rel-phyla-table_L5.qza as BIOMV210DirFmt to directory roo5
Exported rel_OTUtables/rel-phyla-table_L6.qza as BIOMV210DirFmt to directory roo6


In [134]:
!rm -r roo1
!rm -r roo2
!rm -r roo3
!rm -r roo4
!rm -r roo5
!rm -r roo6